We are using sqlite3 for the SQLite queries.

In [44]:
import sqlite3

db = sqlite3.connect("my_db.sqlite") # This creates the files if doesn't exist

We run the table creation script and list all available tables.

In [45]:
# Open file and run script
with open("tables.sql", "r") as file:
    query = file.read()
    db.executescript(query)

# Show the list of tables
cursor = db.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('User',), ('Class',), ('Enrollment',), ('Session',), ('Document',), ('Artwork',), ('Cartel',), ('Card',), ('SessionCards',)]


We run the insertion script to populate our tables.

In [46]:
# Open file and run script
with open("inserts.sql", "r") as file:
    query = file.read()
    db.executescript(query)

Then we check if the insertion script worked.

In [47]:
cursor = db.cursor()

cursor.execute("SELECT * FROM User")
print(cursor.fetchall())

cursor.execute("SELECT * FROM Class")
print(cursor.fetchall())

cursor.execute("SELECT * FROM Enrollment")
print(cursor.fetchall())

cursor.execute("SELECT * FROM Session")
print(cursor.fetchall())

cursor.execute("SELECT * FROM Document")
print(cursor.fetchall())

cursor.execute("SELECT * FROM Artwork")
print(cursor.fetchall())

cursor.execute("SELECT * FROM Cartel")
print(cursor.fetchall())

cursor.execute("SELECT * FROM Card")
print(cursor.fetchall())

cursor.execute("SELECT * FROM SessionCards")
print(cursor.fetchall())


[(1, 0, 'seraphin', 'perrot', 'seraphin.perrot@epitech.eu', 'PwInClearAreABadHabbitLol', 1, '2022-12-06 12:16:16', None, None, None, None), (2, 1, 'mehdi', 'pirahandeh', 'mehdi.pirahandeh@inha.kr', 'Password', 2, '2022-12-06 12:16:16', None, None, None, None)]
[(1, 'Database', 2, 1, '2022-12-06 12:16:16', None, None, None, None), (2, 'Embedded System', 2, 1, '2022-12-06 12:16:16', None, None, None, None)]
[(1, 1, 1, 1, '2022-12-06 12:16:16', None, None, None, None)]
[(1, 'Week 1', 1, 1, '2022-12-06 12:16:16', None, None, None, None), (2, 'Week 1', 2, 1, '2022-12-06 12:16:16', None, None, None, None), (3, 'Week 2', 1, 1, '2022-12-06 12:16:16', None, None, None, None)]
[(1, '~/Documents/Ukiyo-e/TheGreatWaveOffKanagawa.png', 1, 1, '2022-12-06 12:16:16', None, None, None, None)]
[(1, '~/Artworks/Ukiyo-e/TheGreatWaveOffKanagawa.png', 1, '2022-12-06 12:16:16', None, None, None, None)]
[(1, 'katsushika hokusai', 'woodblock print', '1831', 'description TEXT', 1, '2022-12-06 12:16:16', None, No

Now let's try some queries, first let's try to query our one to many class-session relationship.

In [48]:
# We query all our Session associated with the 'Database' class
cursor.execute('''
    SELECT c.* 
    FROM Class c
    JOIN User u ON c.teacher_id = u.id
    WHERE u.last_name = "pirahandeh"
''')
print(cursor.fetchall())

[(1, 'Database', 2, 1, '2022-12-06 12:16:16', None, None, None, None), (2, 'Embedded System', 2, 1, '2022-12-06 12:16:16', None, None, None, None)]


Now let's try to query a many to many relationship, the sessions and cards.

In [49]:
# We query all cards associated with the session with id 1
cursor.execute('''
    SELECT c.*
    FROM Card c
    JOIN SessionCards sc ON c.id = sc.card_id
    JOIN Session s ON sc.session_id = s.id
    WHERE s.id = 1
''')
print(cursor.fetchall())

[(1, 1, 1, 1, '2022-12-06 12:16:16', None, None, None, None), (2, 1, 1, 1, '2022-12-06 12:16:16', None, None, None, None)]


Ok so it worked but we had to provide the id of the session directly, let's say we want all Week 1 cards from the Database class, but we cannot query by id.

In [50]:
# We query all cards where the associated session name is "Week 1" and the associated Class is "Database"
cursor.execute('''
    SELECT c.*
    FROM Card c
    JOIN SessionCards sc ON c.id = sc.card_id
    JOIN Session s ON sc.session_id = s.id
    WHERE s.session_name = "Week 1" AND s.class_id = (
        SELECT id FROM Class WHERE class_name = "Database"
    )
''')
print(cursor.fetchall())

[(1, 1, 1, 1, '2022-12-06 12:16:16', None, None, None, None), (2, 1, 1, 1, '2022-12-06 12:16:16', None, None, None, None)]
